# introduction / example

In this post we explain certain tools to investigate how an optimal solution changes when the input data changes. First we will introduce the notions and study how to do it in pyomo with the help of a small example.

One usage of these tool could be: Suppose we have factories with certain capacities producing certain products. We used a linear program to derive an optimal solution how to use our resources best. In this optimal setting some factories will running at their capacity limit.
Now suppose we want to produce/sell more, hence we have to increase the capacities. Hence a natural question is which capacities should be increased first. Of course we could do simmulations using our linear program, but with the help of what we will see soon these information are allready available with our optimal solution.


## content / dropwords

- [pyomo suffixes](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html)
- sensitivity analysis
- accessing duals
- reduced costs
- shadow prices
- slack values

# "Definitions"

Suppose we like to solve a linear program in following standard form
$$
\begin{array}{ll}
\max & c \cdot x \\
s.t. & Ax \leq b
\end{array}
$$

We can transform the inequalities into equalities using *slack variables* $z$:
$$
Ax +z = b
$$

When the simplex method is used to
solve a linear program, it calculates an optimal solution, an optimal objective function value, and partitions the variables into basic variables and nonbasic variables.

Simply said the optimal solution divides the variables into 2 types:
- *basic variable*, which are between their bounds and
- *non basic variable*, which are allways at their bounds.


# Example

We use the following example from [@bisschop2006aimms, chapter 2.1.2] to show how 

Suppose we are producing 2 types of chips: plain chips and Mexican chips. Both chips go through three main processes, namely slicing, frying, and packing. These processes have the following time characteristics:

- Mexican chips are sliced with a serrate knife, which takes more time than slicing plain chips.
- Frying Mexican chips also takes more time than frying plain chips because of their shape.
- The packing process is faster for Mexican chips because these are only sold in one kind of bag, while plain chips are sold in both family-bags and smaller ones.

There is a limit on the amount of time available for each process because
the necessary equipment is also used for other purposes.The chips also have
different contributions to net profit.

For simplicity we assumed that the market absorbs all produced chips at the fixed price.
The planner of the company now has to determine a production plan that
yields maximum net profit, while not violating the constraints described above.

# algebraic description

## sets

- $type$ - chip type
- $process$ - production process

## variables

- $x_i$ produced amount of chip type $i \in type$

## parameters

- $profit_i$ - profit of 1 kg of type $i$ chip in \$
- $avail_j$ - available time for production process $j$
- $time_{ij}$ - required time in min for 1 kg of type $i$ chip and production process $j$

## constraints

- (c1) time restriction on slicing
- (c2) time restriction on frying
- (c3) time restriction on packing
- production quantities are not negative

## objective

Maximize net profit

## model

$$
\begin{array}{lll}
\max & \sum_{i\in type} profit_i \cdot x_i & \\
s.t. & \sum_{i\in type} time_{ij} \cdot x_i \leq avail_j & \forall j\in process \\
     & x_{i} \leq 0 & 
\end{array}
$$

In [ ]:
import pyomo.environ as pyo
import json

In [ ]:
data = {
    "Name": "Chip Shop",
    "constraints": {
        'slicing': {'plain': 2, 'mexican': 4, 'availibility': 345},
        'frying': {'plain': 4, 'mexican': 5, 'availibility': 480},
        'packing': {'plain': 4, 'mexican': 2, 'availibility': 330},
    },
    "profit": {'plain': 2., 'mexican': 1.5},
    "Engine": "cbc",
    "TimeLimit": ""

}

## Interlude accessing duals in Pyomo

In order to extract the desired information we are using [suffixes in pyomo](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html). Suffixes provide a mechanism for declaring extraneous model data, which can be used in a number of contexts. The following code snippet shows how to declare a suffix component:

`m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)`

## chips model

In [ ]:
def chip_shop(data):    
    m = pyo.ConcreteModel(data['Name'])
    
    #sets
    m.types = pyo.Set(initialize = list(data['profit'].keys()))
    m.process = pyo.Set(initialize = list(data['constraints'].keys()))
    
    # decision variables
    m.x = pyo.Var(m.types, domain = pyo.NonNegativeReals, doc = 'produced packages of chip type i')
    
    
    
    # parameter
    @m.Param(m.types, m.process, doc = 'processing time of product i in process j')
    def time(m,i,j):
        return data['constraints'][j][i]
    @m.Param(m.process, doc = 'available processing time for process j')
    def avail(m,j):
        return data['constraints'][j]['availibility']
    @m.Param(m.types, doc = 'net profits for product i')
    def profit(m,i):
        return data['profit'][i]
    
    # objective
    m.OBJ = pyo.Objective(expr = pyo.quicksum(m.profit[i] * m.x[i] for i in m.types),
                         sense = pyo.maximize)
    
    # constraints
    @m.Constraint(m.process)
    def c(m,j):
        return pyo.quicksum(m.time[i,j] * m.x[i] for i in m.types) <= m.avail[j]
    
    # declaring a Suffix component - to access duals, slack, etc.
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    
    #extra components for output
    m.optimaltime = pyo.Var(m.process, domain = pyo.NonNegativeReals)
    @m.Constraint(m.process)
    def opttime(m,j):
        return m.optimaltime[j] == pyo.quicksum(m.time[i,j] * m.x[i] for i in m.types)
    
    # choosing and applying a solver
    solver = pyo.SolverFactory('cbc')
    solver.solve(m)
    
    return m

In [ ]:
m = chip_shop(data)
print('objective value: %s' % pyo.value(m.OBJ))
for i in m.types:
    print('%10s optimal production: %3s' % (i, pyo.value(m.x[i])))

objective value: 190.0
     plain optimal production: 57.5
   mexican optimal production: 50.0


# Shadow prices

The *marginal value of a constraint, called its shadow price,* is defined as the rate of change of the objective function when increasing the right-hand side of the constraint by one unit.

- A positive shadow price indicates that the objective will increase with a unit increase of the right hand side, while a negative shadow price indicates that the objective will decrease.
- A nonbinding constraint will have a zero shadow price, as its right hand side is not constraining the optimal solution.

This is of course intuitivly clear as relaxing a binding constraint, will enlarge the feasible region. 

In [ ]:
m.pprint()

3 Set Declarations
    process : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'slicing', 'frying', 'packing'}
    time_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain        : Size : Members
        None :     2 : types*process :    6 : {('plain', 'slicing'), ('plain', 'frying'), ('plain', 'packing'), ('mexican', 'slicing'), ('mexican', 'frying'), ('mexican', 'packing')}
    types : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'plain', 'mexican'}

3 Param Declarations
    avail : available processing time for process j
        Size=3, Index=process, Domain=Any, Default=None, Mutable=False
        Key     : Value
         frying :   480
        packing :   330
        slicing :   345
    profit : net profits for product i
        Size=2, Index=types, Domain=Any, Default=None, Mutable=False
        Key     :

### Example: shadow prices for chips production



In [ ]:
print("printing duals")
for c in m.component_objects(pyo.Constraint, active=True):
    print("   Constraint", c)
    for index in c:
        print("      ", index, m.dual[c[index]])

printing duals
   Constraint c
       slicing -0.0
       slicing -0.0
       frying 0.16666667
       frying 0.16666667
       packing 0.33333333
       packing 0.33333333
   Constraint opttime
       slicing -0.0
       slicing -0.0
       frying -0.0
       frying -0.0
       packing -0.0
       packing -0.0


In [ ]:
print("Constraint  value  lslack  uslack    dual")
for c in [model.demand, model.laborA, model.laborB]:
    print(c, str.format(c(), c.lslack(), c.uslack(), model.dual[c]))


In [ ]:
for c in m.component_objects(pyo.Constraint, active=True):
    print(c, str.format(c(), c.lslack(), c.uslack(), m.dual[c]))

TypeError: 'IndexedConstraint' object is not callable

In [ ]:
{
    c: {i:{'process':i,
           'optimal time': pyo.value(m.optimaltime[i]),
           'upper bound': pyo.value(m.c[i].upper),
           'shadow price': m.dual[c[index]]
          }
        for i in c        
       }
    for c in m.component_objects(pyo.Constraint, active=True)  
}

{<pyomo.core.base.constraint.IndexedConstraint>: {'slicing': {'process': 'slicing',
   'optimal time': 315.0,
   'upper bound': 345.0,
   'shadow price': 0.33333333},
  'frying': {'process': 'frying',
   'optimal time': 480.0,
   'upper bound': 480.0,
   'shadow price': 0.33333333},
  'packing': {'process': 'packing',
   'optimal time': 330.0,
   'upper bound': 330.0,
   'shadow price': 0.33333333}},
 <pyomo.core.base.constraint.IndexedConstraint>: {'slicing': {'process': 'slicing',
   'optimal time': 315.0,
   'upper bound': 345.0,
   'shadow price': -0.0},
  'frying': {'process': 'frying',
   'optimal time': 480.0,
   'upper bound': 480.0,
   'shadow price': -0.0},
  'packing': {'process': 'packing',
   'optimal time': 330.0,
   'upper bound': 330.0,
   'shadow price': -0.0}}}